In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import itertools

import os
os.sys.path.append('../')
import utilities as u
import preprocessing as pp


%load_ext autoreload
%autoreload 2

%matplotlib inline

In [32]:
mouse = '4139265.4'
df = pp.load_session_db(dir=os.path.join("/Volumes","GoogleDrive","My Drive"))

df = df[df['RewardCount']>30]
df = df[df['Imaging']==1]
df = df.sort_values(['MouseName','DateTime','SessionNumber'])
tracks = 'TwoTower_noTimeout|TwoTower_Timeout|Reversal_noTimeout|Reversal|TwoTower_foraging'
df = df[df['Track'].str.contains(tracks,regex=True)]

df = df.sort_values(['MouseName','DateTime','SessionNumber'])

df_mouse = df[df['MouseName'].str.match(mouse)]
df_mouse

/Volumes/GoogleDrive/My Drive/VR_Data/TwoTower/behavior.sqlite


,MouseName,DateFolder,SessionNumber,Track,RewardCount,Imaging,ImagingRegion,DateTime,data file,scanfile,scanmat,s2pfolder
1371,4139265.4,07_02_2019,2,TwoTower_foraging,60,1,CA1,2019-02-07,/Volumes/GoogleDrive/My Drive/VR_Data/TwoTower...,None,/Volumes/GoogleDrive/My Drive/2P_Data/TwoTower...,/Volumes/GoogleDrive/My Drive/2P_Data/TwoTower...
1373,4139265.4,07_02_2019,4,TwoTower_foraging,60,1,CA1,2019-02-07,/Volumes/GoogleDrive/My Drive/VR_Data/TwoTower...,None,/Volumes/GoogleDrive/My Drive/2P_Data/TwoTower...,/Volumes/GoogleDrive/My Drive/2P_Data/TwoTower...
1407,4139265.4,10_02_2019,2,TwoTower_foraging,60,1,CA1,2019-02-10,/Volumes/GoogleDrive/My Drive/VR_Data/TwoTower...,None,/Volumes/GoogleDrive/My Drive/2P_Data/TwoTower...,/Volumes/GoogleDrive/My Drive/2P_Data/TwoTower...
1436,4139265.4,12_02_2019,2,TwoTower_foraging,85,1,CA1,2019-02-12,/Volumes/GoogleDrive/My Drive/VR_Data/TwoTower...,None,/Volumes/GoogleDrive/My Drive/2P_Data/TwoTower...,/Volumes/GoogleDrive/My Drive/2P_Data/TwoTower...
1459,4139265.4,15_02_2019,3,TwoTower_foraging,82,1,CA1,2019-02-15,/Volumes/GoogleDrive/My Drive/VR_Data/TwoTower...,None,/Volumes/GoogleDrive/My Drive/2P_Data/TwoTower...,/Volumes/GoogleDrive/My Drive/2P_Data/TwoTower...
1478,4139265.4,17_02_2019,2,TwoTower_foraging,94,1,CA1,2019-02-17,/Volumes/GoogleDrive/My Drive/VR_Data/TwoTower...,None,/Volumes/GoogleDrive/My Drive/2P_Data/TwoTower...,/Volumes/GoogleDrive/My Drive/2P_Data/TwoTower...
1489,4139265.4,18_02_2019,2,TwoTower_foraging,101,1,CA1,2019-02-18,/Volumes/GoogleDrive/My Drive/VR_Data/TwoTower...,None,/Volumes/GoogleDrive/My Drive/2P_Data/TwoTower...,/Volumes/GoogleDrive/My Drive/2P_Data/TwoTower...
1498,4139265.4,19_02_2019,2,TwoTower_foraging,97,1,CA1,2019-02-19,/Volumes/GoogleDrive/My Drive/VR_Data/TwoTower...,None,/Volumes/GoogleDrive/My Drive/2P_Data/TwoTower...,/Volumes/GoogleDrive/My Drive/2P_Data/TwoTower...
1510,4139265.4,20_02_2019,2,TwoTower_foraging,99,1,CA1,2019-02-20,/Volumes/GoogleDrive/My Drive/VR_Data/TwoTower...,None,/Volumes/GoogleDrive/My Drive/2P_Data/TwoTower...,/Volumes/GoogleDrive/My Drive/2P_Data/TwoTower...


In [ ]:
sess = df_mouse.iloc[-1]
VRDat,C,S,A = pp.load_scan_sess(sess)

num aberrant ttls 0.0


In [4]:
avec = np.linspace(0,1,num=20)
C = np.array([(x,y) for x,y in itertools.product(avec.tolist(),avec.tolist())])
dispersion = .01
beta = np.ones([C.shape[0],])
rbf = lambda x : np.exp(-np.diag(np.matmul(x-C,(x-C).T))/dispersion)
K = lambda X: np.array(list(map(rbf,X.tolist())))
mu = lambda X,b: np.exp(np.dot(K(X),b))

In [13]:
A  = np.array([[0,1],[2,3],[4,5]])
beta = np.random.randn(400)

print(K(A).shape,beta.shape,mu(A,beta).shape)

(3, 400) (400,) (3,)


In [14]:
def objective(S,X,b,alpha=.1):
    return -(S*mu(X,b) - np.exp(mu(X,b))).sum() + alpha*np.linalg.norm(b,ord=2)

def gradient(S,X,b,alpha=.1):
    # return gradient vector
    return -((S-np.exp(mu(X,b)))*K(X)).sum(axis=0) +2*alpha*b
    

def hessian(S,X,b,alpha=.1):
    H = np.zeros([b.size,b.size])
    D = 0
    for i in range(S.shape[0]):
        u = np.exp(mu(X[i,:],b))*K(X[i,:])
        v = np.copy(K(X[:,i]))
        D+=np.matmul(u[:,np.newaxis],v[np.newaxis,:])
        
    D+= 2*alpha*np.eye(D.shape[0])
        
    return D

SyntaxError: invalid syntax (<ipython-input-14-b41a2d8d06dd>, line 13)